In [5]:
import pandas as pd
data = pd.read_csv('train.csv')
#print(data)

print(data.isnull().sum())

avg = data['Age'].mean()
print(avg)
em = data['Embarked'].mode()


data['Age'].fillna(value=30, inplace=True)
data['Embarked'].fillna(value='S', inplace=True)

print(data.isnull().sum())



PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64
29.69911764705882
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


In [6]:
import tensorflow as tf

result = data.pop('Survived')

# 데이터 셋 만들기 (X데이터, 정답)
ds = tf.data.Dataset.from_tensor_slices((dict(data), result))
print(ds)


<_TensorSliceDataset element_spec=({'PassengerId': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Pclass': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Name': TensorSpec(shape=(), dtype=tf.string, name=None), 'Sex': TensorSpec(shape=(), dtype=tf.string, name=None), 'Age': TensorSpec(shape=(), dtype=tf.float64, name=None), 'SibSp': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Parch': TensorSpec(shape=(), dtype=tf.int64, name=None), 'Ticket': TensorSpec(shape=(), dtype=tf.string, name=None), 'Fare': TensorSpec(shape=(), dtype=tf.float64, name=None), 'Embarked': TensorSpec(shape=(), dtype=tf.string, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [ ]:
def nomalizer(x):
    mini = data['Fare'].min()
    maxi = data['Fare'].max()
    return (x-mini) / (maxi - mini)
    

In [24]:
feature_columns = []

# 숫자로 집어넣어야 하는 것 Fare Parch SibSp : numeric_column
# 뭉퉁그려서 집어넣을 것 Age : bucketized_column
# 종류 몇개없는 카테고리화해서 접어넣어야 하는 것 Sex Embarked Pclass : indicator_column
# 종류가 많은 카테고리 Ticket : embedding_column

feature_columns.append(tf.feature_column.numeric_column('Fare'))
feature_columns.append(tf.feature_column.numeric_column('Parch'))
feature_columns.append(tf.feature_column.numeric_column('SibSp'))
# feature_columns.append(tf.feature_column.numeric_column('Age'))

Age = tf.feature_column.numeric_column('Age')
Age_bucket = tf.feature_column.bucketized_column(Age, boundaries=[10,20,30,40,50,60])
feature_columns.append(Age_bucket)

print(Age)
print(feature_columns)


NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
[NumericColumn(key='Fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SibSp', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), BucketizedColumn(source_column=NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(10, 20, 30, 40, 50, 60))]


In [25]:
# 컬럼에 있는 유니크한 리스트
vocab = data['Sex'].unique()

# 원 핫 인코등
cate = tf.feature_column.categorical_column_with_vocabulary_list('Sex',vocab )
# print(vocab)
one_hot = tf.feature_column.indicator_column(cate)
feature_columns.append(one_hot)

# 컬럼에 있는 유니크한 리스트
vocab = data['Embarked'].unique()
# 원 핫 인코등
cate = tf.feature_column.categorical_column_with_vocabulary_list('Embarked',vocab )
# print(vocab)
one_hot = tf.feature_column.indicator_column(cate)
feature_columns.append(one_hot)

# 컬럼에 있는 유니크한 리스트
vocab = data['Pclass'].unique()
# 원 핫 인코등
cate = tf.feature_column.categorical_column_with_vocabulary_list('Pclass',vocab )
# print(vocab)
one_hot = tf.feature_column.indicator_column(cate)
feature_columns.append(one_hot)

# embedding
vocab = data['Ticket'].unique()
# 원 핫 인코등
cate = tf.feature_column.categorical_column_with_vocabulary_list('Ticket',vocab )
# print(vocab)
embedding = tf.feature_column.embedding_column(cate, dimension=9)
feature_columns.append(embedding)



In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
ds_batch = ds.batch(32)
print(ds_batch)
print(feature_columns)
model.fit(ds_batch, shuffle=True, epochs=100)

<_BatchDataset element_spec=({'PassengerId': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Pclass': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Name': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Sex': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Age': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'SibSp': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Parch': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Ticket': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Fare': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Embarked': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
[NumericColumn(key='Fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SibSp', shape=(1,), default_value=None, dtype=tf.float32